In [ ]:
!pip install -U oci

In [11]:
import requests
import pandas as pd
import re
import oci
import io
import ocifs
import os
import base64
import numpy as np 

from PIL import Image
from ads.dataset.factory import DatasetFactory
from ads.common.auth import default_signer


In [2]:
config = oci.config.from_file('~/.oci/config')
object_storage_client = oci.object_storage.ObjectStorageClient(config)

In [3]:
ai_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)

In [4]:
fs = ocifs.OCIFileSystem(config="~/.oci/config")

img_list = fs.glob("oci://<nome-do-bucket>@<namespace>/*")

In [32]:
name_list = []
for i, item in enumerate(img_list):
    name = item.rsplit('/', 1)[1]
    name_list.append(name)
    
code_list = []
for i in range(0, len(img_list)):
    codigo = img_list[i]
    s = [float(s) for s in re.findall(r'-?\d+\.?\d*', codigo)]
    code_list.append(int(s[1]))

In [ ]:
for i, file in enumerate(img_list):
    with fs.open(file) as f:
            content = f.read()
            
            encoded_string = base64.b64encode(content, altchars=None)
            decoded_string = encoded_string.decode("utf-8", "ignore")

            analyze_document_response = ai_vision_client.analyze_document(
                    analyze_document_details=oci.ai_vision.models.AnalyzeDocumentDetails(
                    compartment_id = "ocid1.compartment.oc1..aaaaaaaaxxxxxxxxxxxxxxxxxx5xxxxxxxxxxxxxxxxx6xxxxxxxx",
                    features=[
                        oci.ai_vision.models.DocumentTextDetectionFeature(
                            feature_type="TEXT_DETECTION")],
                    language='ENG',
                    document=oci.ai_vision.models.InlineDocumentDetails(
                    source="INLINE",
                    data=decoded_string)))

            words = str([word.text for page in analyze_document_response.data.pages for word in page.words])
            name = f"{name_list[i]}.txt"  
            
            words_str_1 = words.replace("',",'')
            words_str_2 = words_str_1.replace("'",'')
            words_str_3 = words_str_2.replace("[",'')
            words_str_4 = words_str_3.replace("]",'')
            
            put_object_response = object_storage_client.put_object(
                                        namespace_name='<namespace>',
                                        bucket_name='<nome-do-bucket>',
                                        object_name=name,
                                        put_object_body = words_str_4,
                                        content_type="text/plain")

            print("Documento", name, "upado!")